In [ ]:
typeDict = IfcTypeDict()

assert typeDict["IfcWall"] == (
    'GlobalId',
    'OwnerHistory',
    'Name',
    'Description',
    'ObjectType',
    'ObjectPlacement',
    'Representation',
    'Tag')

nodes = []
edges = []

ourLabel = 'test'

f = IfcOpenShell.open(ifc_path)

for el in f:
    if el.is_a() == "IfcOwnerHistory":
        continue
    tid = el.id()
    cls = el.is_a()
    pairs = []
    keys = []
    try:
        keys = [x for x in el.get_info() if x not in ["type", "id", "OwnerHistory"]]
    except RuntimeError:
        # we actually can't catch this, but try anyway
        pass
    for key in keys:
        val = el[key]
        if any(hasattr(val, "is_a") and val.is_a(thisTyp)
               for thisTyp in ["IfcBoolean", "IfcLabel", "IfcText", "IfcReal"]):
            val = val.wrappedValue
        if val and type(val) is tuple and type(val[0]) in (str, bool, float, int):
            val = ",".join(str(x) for x in val)
        if type(val) not in (str, bool, float, int):
            continue
        pairs.append((key, val))

    nodes.append((tid, cls, pairs))
    for i in range(len(el)):
        try:
            el[i]
        except RuntimeError as e:
            if str(e) != "Entity not found":
                print("ID", tid, e, file=sys.stderr)
            continue
        if isinstance(el[i], IfcOpenShell.entity_instance):
            if el[i].is_a() == "IfcOwnerHistory":
                continue
            if el[i].id() != 0:
                edges.append((tid, el[i].id(), typeDict[cls][i]))
                continue
        try:
            iter(el[i])
        except TypeError:
            continue
        destinations = [
            x.id() for x in el[i] if isinstance(
                x, IfcOpenShell.entity_instance)]
        for connectedTo in destinations:
            edges.append((tid, connectedTo, typeDict[cls][i]))
            
if len(nodes) == 0:
    print("no nodes in file", file=sys.stderr)
    sys.exit(1)

indexes = set(["nid", "cls"])

In [ ]:
import itertools
import IfcOpenShell
import sys
from py2neo import Graph, Node
import time


def chunks2(iterable, size, filler=None):
    it = itertools.chain(iterable, itertools.repeat(filler, size - 1))
    chunk = tuple(itertools.islice(it, size))
    while len(chunk) == size:
        yield chunk
        chunk = tuple(itertools.islice(it, size))


class IfcTypeDict(dict):
    def __missing__(self, key):
        value = self[key] = IfcOpenShell.create_entity(
            key).wrapped_data.get_attribute_names()
        return value

Error: Error: Jupyter cannot be started. Error attempting to locate jupyter: c:\_Python\IFC-Neo4j-converter\.pyenv\Scripts\python.exe: Error while finding spec for 'notebook.__main__' (<class 'ImportError'>: cannot import name 'Type'); 'notebook' is a package and cannot be directly executed
